In [47]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

df = pd.read_csv('FormattedData/train.csv', sep = ',' , header=0)

df.head()

TRAIN_SPLIT = 2000

tf.random.set_seed(13)

#Which features in dataset to consider
features_considered = ['MON1 CL DDM (?A)', 'MON1 CL RF (dB)', 'MON1 DS DDM (?A)', 'MON1 DS RF (dB)', 'MON1 NF DDM (?A)', 'MON1 NF RF (dB)' , 'MON1 CLR DDM (?A)']

features = df[features_considered]
features.index = df['Timestamp']
features.head()

features.plot(subplots=True)


TypeError: datetime64[ns] cannot be converted to a FloatingDtype

In [44]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset.iloc[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data).astype(np.float32), np.array(labels).astype(np.float32)

In [45]:
past_history = 500
future_target = 72
STEP = 6

x_train_multi, y_train_multi = multivariate_data(df, df.iloc[:, 1], 0, TRAIN_SPLIT, past_history, future_target, STEP)

x_val_multi, y_val_multi = multivariate_data(df, df.iloc[:, 1], TRAIN_SPLIT, None, past_history, future_target, STEP)

TypeError: float() argument must be a string or a number, not 'Timestamp'

In [36]:
BATCH_SIZE = 256
BUFFER_SIZE = 10000

train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).